# Display Guild Lineage Tables

In [1]:
import os, sys
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
os.path.exists(DATA_PATH)

True

In [4]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [7]:
import engine.models as GM
import engine.logic.resource as RS
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['GuildStage', 'ItemInGuild', 'ItemUnlocked', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Tables

In [9]:
userTb = get_user_model()
adminMd = userTb.objects.getUser(email='admin@thieves-guild.com')
adminMd

<User: admin@thieves-guild.com>

In [10]:
guildMds = GM.Guild.objects.filter(UserFK=adminMd)
guildLs = list(guildMds.values())

if len(guildLs) > 0:
    guildDf = PD.DataFrame(guildLs)
    guildDf = guildDf.drop(['UserFK_id', 'VaultGold', 'VaultGems'], axis=1, errors='ignore')
    guildDf = guildDf.sort_values('LastPlayed', ascending=False)
    guildDf
else:
    print('no guilds')

,id,Name,LastPlayed,CreateDate,Selected,KeepLevel,CampaignWorld,TotalPower,StorageGold,StorageGems,StorageWood,StorageStone,StorageIron,VaultWood,VaultStone,VaultIron
0,10,123,2024-01-01,2024-01-01,True,1,1,672,0,0,0,0,0,0,0,0


In [17]:
guildName = '123'
guildMd = GM.Guild.objects.GetOrNone(Name=guildName)
thiefMds = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
thiefLs = list(thiefMds.values())

thiefDf = PD.DataFrame(thiefLs)
thiefDf = thiefDf.drop(['GuildFK_id', 'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
                        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', ], 
                       axis=1, errors='ignore')
thiefDf = thiefDf.sort_values('Power', ascending=False)
thiefDf

,id,Name,Class,Stars,BasePower,Level,Experience,Power,Agility,Cunning,...,Endurance,Health,Attack,Damage,Defense,Sabotage,Perceive,Traverse,Status,CooldownExpire
0,39,Bellot,Burglar,1,48,1,260,128,5,0,...,0,58,5,7,13,5,0,5,Ready,None
3,42,Sypha,Burglar,1,48,1,260,128,5,0,...,0,58,5,8,14,0,0,0,Ready,None
2,41,Toph,Ruffian,1,48,1,186,112,0,0,...,0,58,3,6,16,0,5,0,Ready,None
4,43,Djahal,Scoundrel,1,48,1,260,112,0,5,...,0,58,3,11,12,0,0,0,Ready,None
1,40,Depingo,Scoundrel,1,48,1,0,96,0,5,...,0,58,2,11,12,0,0,0,Ready,None
5,44,Tekuho,Ruffian,1,48,1,260,72,0,0,...,0,58,3,6,14,0,0,0,Ready,None


In [12]:
assetMds = GM.ItemInGuild.objects.filter(GuildFK=guildMd)
assetLs = list(assetMds.values())

assetDf = PD.DataFrame(assetLs)
assetDf = assetDf.drop(['GuildFK_id', ], axis=1, errors='ignore')
assetDf = assetDf.sort_values('Power', ascending=False)
assetDf

,id,ThiefFK_id,Slot,Name,Level,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic
0,104,42,weapon,Twin Daggers,1,1,24,Burglar,agi 1,None,dmg 1,None
7,111,42,armor,Brigandine,1,1,24,Burglar,agi 1,None,def 3,None
11,115,44,armor,Chain Maille,1,1,24,Ruffian,mig 1,None,def 1,None
10,114,44,weapon,Sword & Shield,1,1,24,Ruffian,mig 1,None,att 3,None
1,105,39,armor,Brigandine,1,1,24,Burglar,agi 1,None,def 3,None
8,112,43,weapon,Blackthorn,1,1,24,Scoundrel,cun 1,None,att 2,None
9,113,43,armor,Vestment,1,1,24,Scoundrel,cun 1,None,def 2,None
6,110,39,weapon,Twin Daggers,1,1,24,Burglar,agi 1,None,dmg 1,None
5,109,41,armor,Chain Maille,1,1,24,Ruffian,mig 1,None,def 1,None
4,108,41,weapon,Sword & Shield,1,1,24,Ruffian,mig 1,None,att 3,None
